In [ ]:
import pandas as pd
from google.colab import drive
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns

In [ ]:
# change the name with the best iteration in your case
best_performing_iteration = 30250
loading_model = torch.load("rnn_lstm_model#"+best_performing_iteration+".pt")

In [ ]:
# load data
test = pd.read_csv(r"/content/drive/MyDrive/ECG Project Files/processed_test.csv", delimiter=",")

# split data into features(pixels) and labels(numbers from 0 to 9)
targets_numpy = test.Label.values
features_numpy = test.loc[:,test.columns != "Label"].values

features_numpy_length = features_numpy.shape[0]
processed_data = []
processed_labels = []

for i in range (features_numpy_length):
  try:
    temp_array = features_numpy[i][0].split(',')
    temp2_array = []
    for j in range(len(temp_array)):
      temp2_array.append(float(temp_array[j]))
    if(len(temp2_array)<401):
      processed_data.append(temp2_array)
      processed_labels.append(targets_numpy[i])

  except ValueError:
    print("Line # that contains an unexpected character:", i)
    continue

padded_data = np.stack([np.pad(i,( int((400-len(i))/2) , 400-int((400-len(i))/2)-len(i) ),mode='edge') for i in processed_data])

features_numpy = np.array(padded_data)
targets_numpy = np.array(processed_labels)

print(features_numpy.shape)
print(targets_numpy.shape)

In [ ]:
featuresTest = torch.from_numpy(features_numpy)
targetsTest = torch.from_numpy(targets_numpy)
new_data_test = TensorDataset(featuresTest,targetsTest)
new_data_test_loader = DataLoader(new_data_test, shuffle = True)

In [ ]:
total = 0
correct = 0
count = 0
predictions_list = []
labels_list = []

for signals, labels in new_data_test_loader:
    #############################################
    #To Enable GPU Usage
    if use_cuda and torch.cuda.is_available():
      signals = signals.cuda()
      labels = labels.cuda()
    #############################################
    signals = Variable(signals.float().view(-1, 400, 1))
    outputs = loading_model(signals)
    max = -1
    predicted = -1
    for i in range(len(outputs.data)):
      if (outputs.data[0][i] > max)and((i==0)or(i==3)or(i==5)):
        max = outputs.data[0][i]
        predicted = i
    predictions_list.append(predicted)
    labels_list.append(labels)
    total += float(labels.size[0])
    count += 1
    if (predicted == labels):
      correct += 1
accuracy = 100 * correct / float(total)
print("Accuracy of the model on the test dataset: ", accuracy)